# Coletando dados da página de funcionários de uma empresa no linkedin

Neste projeto, iremos utilizar o selenium para coletar dados da página de funcionários de uma empresa no linkedin. No exemplo, a empresa em questão é a ClearSale, empresa em que trabalho como Especialista de People Analytics, e os dados que iremos coletar serão:


*   Nome do Perfil
*   URL do Perfil
*   Localização do Perfil
*   Títulos do Perfil



## Instalando e Importando as Dependências:

In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from getpass import getpass
from time import sleep
import pandas as pd

## Abrindo o Navegador:

*   É necesário fazer o download do Chromedriver com a versão compatível do seu navegador e deixá-lo no mesmo path do arquivo que deseja executar.
*   Caso não teha feito o download, este é o link : https://chromedriver.chromium.org/downloads

In [ ]:
driver = webdriver.Chrome(r"C:\Users\Romulo\Desktop\chromedriver.exe")

Neste momento, iremos pedir que o navegador acesse a URL do Linkedin:

In [ ]:
url = 'http://www.linkedin.com'

In [ ]:
driver.get(url)

## Instanciando Usuário e Senha:

Vamos utilizar a biblioteca getpass para manter o usuário e a senha. Se precisar escrever um programa que leia do teclado a senha do usuário e não quer que ela seja ecoada, isto é, não seja visível, use o módulo getpass.

In [ ]:
usermail = getpass()

In [ ]:
senha = getpass()

## Preenchendo o Formulário de Login:

In [ ]:
# Preenchendo o usuário
element_login = driver.find_element_by_xpath('//*[@id="session_key"]')
element_login.clear()
element_login.send_keys(usermail)

sleep(2)

# Preenchendo a senha
element_senha = driver.find_element_by_xpath('//*[@id="session_password"]')
element_senha.clear()
element_senha.send_keys(senha)

sleep(2)

# Clicar para entrar no linkedin
driver.find_element_by_xpath('//*[@id="main-content"]/section[1]/div/div/form/button').click()

## Rodando o Crawler:

Para entender como funciona o Crawler, é necessário entender os elementos da página.

Ao entrarmos na página de funcionários da empresa, notamos que podem exister até 100 páginas (limitação do linkedin) com até 10 resultados de pesquisa (funcionários) por página.

Sendo assim, é necessário que (1) o Crawler percorra todas as páginas e (2) percorra todos os resultados de cada página.

Para fazer isso, utilizaremos 2 laços `for`:

*   O primeiro laço é utilizado para alterar as paginas em um range de 1 a 101 (limite de páginas de pesquisa). Note que, na URL, existe um elemento `&page=` seguido de uma string(i), que será o elemento iterado no laço for. Isto é o que vai permitir mudarmos as páginas em cada iteração.

*   O seguindo laço é necessário para alterar os 'cards' de cada funcionário. A iteração acontece diretamente dentro do XPATH, exatamente no elemento que altera entre um card e outro.

*   Utilizamos ainda TRY e EXCEPT para, caso haja algum erro, o laço não seja quebrado e continue para a próxima iteração.



In [ ]:
nomes = []
titulos = []
cidades = []
urls = []

for i in range (1,101):
    driver.get('https://www.linkedin.com/search/results/people/?currentCompany=%5B"202691"%5D&origin=COMPANY_PAGE_CANNED_SEARCH&page='+str(i)+'&sid=38%3A')
    for j in range (1,11):
        try:
            nomes.append(driver.find_element_by_xpath('//*[@id="main"]/div/div/div/ul/li['+str(j)+']/div/div/div[2]/div[1]/div[1]/div/span[1]/span/a/span/span[1]').text)
            titulos.append(driver.find_element_by_xpath('//*[@id="main"]/div/div/div/ul/li['+str(j)+']/div/div/div[2]/div[1]/div[2]/div/div[1]').text)
            cidades.append(driver.find_element_by_xpath('//*[@id="main"]/div/div/div/ul/li['+str(j)+']/div/div/div[2]/div[1]/div[2]/div/div[2]').text)
            urls.append(driver.find_element_by_xpath('//*[@id="main"]/div/div/div/ul/li['+str(j)+']/div/div/div[2]/div[1]/div[1]/div/span[1]/span/a').get_attribute('href'))
        except:
            nomes.append('N/A')
            titulos.append('N/A')
            cidades.append('N/A')
            urls.append('N/A')

    sleep(5)
    

## Criando o Dataframe

Por fim, utilizamos as listas criadas no passo anterior para criar e exportar um dataframe em CSV, para ser utilizado em outras ferramentas, como Excel ou Power BI.

In [ ]:
frame = list(zip(nomes, titulos, cidades, urls))

In [ ]:
df = pd.DataFrame(frame, columns = ['Nomes', 'Titulos', 'Cidades', 'URLs_Perfil'])

In [ ]:
df.to_csv('lista-de-perfis.csv', index=False)

In [ ]:
# Fechando o navegador

driver.quit()